# K-Nearest Neighbors Analysis
Imports and feature selection

In [ ]:
from data_cleaning import ratings_df, watchlist_df, valid_int
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from mlxtend.feature_selection import SequentialFeatureSelector as SFS 
from mlxtend.plotting import plot_sequential_feature_selection


#? Model Training & Feature Selection: Wrapper - Sequential Backwards Selection (SBS)
X = ratings_df[["IMDb Rating", "Runtime (mins)", "Year", "Num Votes", "Day_Rated"]] 
y = ratings_df['Your Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
                                            
model = KNeighborsRegressor(n_neighbors=25, weights = "distance") 

sbs = SFS(model,
          k_features=1, # k=4 for best performance, however setting to 1 so visualsie performance on graph for k 1 through to 5
          forward=False,
          floating=False,
          scoring='r2',
          cv=5)

sbs.fit(X_train_scaled, y_train)

X_train_selected = sbs.transform(X_train_scaled)
X_test_selected = sbs.transform(X_test_scaled)

#? Model 1 & 2
model.fit(X_train_scaled, y_train)
# model.fit(X_train_selected, y_train) # --- With Feature Selection

#? Feature selection evaluation
# print(sffs.subsets_)
plot_sequential_feature_selection(sbs.get_metric_dict(), kind='std_dev')  

## Model Evaluation

In [ ]:
y_pred = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)
print(f'Model 1) \nMean Absolute Error: {round(mae, 3)}')
train_score = model.score(X_train_scaled, y_train)
print(f'Training R-squared: {round(train_score, 3)}')
test_score = model.score(X_test_scaled, y_test)
print(f'Test R-squared: {round(test_score, 3)}', '\n')

print('Model 2) \nMean Absolute Error: 0.895 ') # (K=4) These values were worked out by using model 2 and replacing X test and train scaled with X test and train selected!
print('Test R-squared: 1.0')
print('Training R-squared: 0.336', '\n')

# print(model.coef_)
# print(model.intercept_) 

## Visualisng Model Performance for different K values

In [ ]:

def best_k(training_data, training_labels, test_data, test_labels):
    r2_scores = []

    for k in range(1, len(training_data)):
        model = KNeighborsRegressor(n_neighbors=k)
        model.fit(training_data, training_labels)
        r2_score = model.score(test_data, test_labels)
        r2_scores.append(r2_score)

    
    best_k_value = np.argmax(r2_scores) + 1 
    best_r2_score = r2_scores[best_k_value - 1]  

    return best_k_value, best_r2_score, r2_scores


k_list = range(1, len(X_train_scaled))
best_k_value, best_r2_score, r2_scores2 = best_k(X_train_scaled, y_train, X_test_scaled, y_test) 


plt.plot(k_list, r2_scores2, color="yellow")
plt.xlabel('k Values')
plt.ylabel('R2 Score')
plt.title('R2 Score vs k Values')
plt.show()

print(f"Best k value: {best_k_value}")
print(f"Best r2_score: {round(best_r2_score, 4)}") 

## Predictions on Watchlist

In [ ]:
#! Making sure the predictions df lines up with ratings_df
watchlist_df2 = watchlist_df.drop("Title", axis=1).copy()
watchlist_df2 = watchlist_df2[["IMDb Rating", "Runtime (mins)", "Year", "Num Votes", "Day_Rated"]]

# Predictions
watchlist_scaled = scaler.transform(watchlist_df2)
predictions = np.round(model.predict(watchlist_scaled), 1) # Rounded to make it a closer resemblence to acutal IMDb Ratings..

pred_titles = list(zip(watchlist_df['Title'], predictions))
sorted_predictions = sorted(pred_titles, key=lambda x: x[1], reverse=True)

for title, rating in sorted_predictions:
    print(f'{title}: {rating}')

## User Input Predictions

In [ ]:
# Some movies that you can try:
# The Godfather, 9.2, 175, 1972, 2000000
# Napoleon, 6.5, 158, 2023, 95000
# Frozen 2, 6.8, 103, 2019, 191000

def user_predict():
    movie_title = input("Enter the name of the movie/ tv show you would like rated: ")
    imdb_rating = valid_int("Enter the IMDb rating: ", 0, 10)
    runtime = valid_int("Enter the runtime in minutes: ", 0, 250)
    year = valid_int("Enter the year of release: ", 1888, 2024)
    num_votes = valid_int("Enter the number of votes on Imdb: ", 0, 2134288)

    new_data = pd.DataFrame({
        'IMDb Rating': [imdb_rating],
        'Runtime (mins)': [runtime],
        'Year': [year],
        'Num Votes': [num_votes],
        'Day_Rated': [3487] # defaulted as present day : 26/1/24
    })

    new_data_scaled = scaler.transform(new_data)
    predicted_rating = np.round(model.predict(new_data_scaled),1)
    print(f'\nPredicted rating for {movie_title}: {predicted_rating[0]}')

#! user_predict() # --------- UNCOMMENT this line to use the predicter function

## Visualisation of Predicted vs Actual Rating

In [ ]:
plt.scatter(y_test, y_pred, alpha=0.4)
plt.xlabel("Ratings: /10")
plt.ylabel("Predicted Ratings: /10")
plt.title("Actual Ratings vs Predicted Ratings")
plt.show() 